# Batch processing

Quick script to export musescore files to musicxml and pdf

In [ ]:
import glob
import os
import subprocess
import yaml
from datetime import datetime
import pandas as pd
import seaborn as sns
import music21
from music21.metadata import Metadata

import sys
sys.path.append('../')
from solmization import solmize

CORPUS_DIR = '/Users/bas/surfdrive/Research/Guidonian Hand/Corpus'
MSCORE = '/Applications/MuseScore 4.app/Contents/MacOS/mscore'
MdB_DIR = f'{CORPUS_DIR}/Marot & de Bèze'
SH_DIR = f'{CORPUS_DIR}/Sternhold & Hopkins'
SMITH_DIR = f'{CORPUS_DIR}/Smith'

## Export museScore to musicxml and pdf

In [ ]:
def export_source(source_dir, target_ext='musicxml', refresh=False):
    files = sorted(glob.glob(f'{source_dir}/musescore/*.mscz'))
    if not os.path.exists(f'{source_dir}/{target_ext}'):
        os.makedirs(f'{source_dir}/{target_ext}')
    for file in files:
        name, _ =  os.path.basename(file).split('.')
        target = f'{source_dir}/{target_ext}/{name}.{target_ext}'
        
        if name.endswith('-draft'):
            continue

        if not os.path.exists(target) or refresh:
            result = subprocess.run([MSCORE, '-o', target, file])
            print(result)

In [ ]:
sources = glob.glob(f'{CORPUS_DIR}/**')
refresh = False
for source_dir in sources:
    export_source(source_dir, target_ext='musicxml', refresh=refresh)
    export_source(source_dir, target_ext='pdf', refresh=refresh)

In [ ]:
goudimel_fn = f'{SMITH_DIR}/musicxml/goudimel.musicxml'
isaac_fn = f'{SMITH_DIR}/musicxml/isaac.musicxml'
smith_files = dict(goudimel=goudimel_fn, isaac= isaac_fn)
target_dir = f'{SMITH_DIR}/evaluation'

for name, source_fn in smith_files.items():
    score = music21.converter.parse(source_fn)
    for part in score.parts:
        id = f'{name}-{part.id.lower()}'


## Evaluate

### Processing code

In [ ]:
def process_evaluation(eval):
    for key in ['correct', 'missing', 'insertion', 'deletion', 'incorrect']:
        eval[key] = eval.get(key, 0)
    eval['num_notes'] = sum(eval.values())
    eval['num_syllables'] = eval['num_notes'] - eval['missing']
    # Don't count missing syllables as a mistake
    eval['accuracy'] = eval['correct'] / eval['num_syllables']
    report = f"Accuracy best solmization: {eval['accuracy']:.0%}"
    errors = [f'{eval[k]} {k}' for k in ['incorrect', 'missing', 'insertion', 'deletion'] if eval[k] > 0]
    if len(errors) > 0: report += f' ({", ".join(errors)})'
    return report

def process_stream(stream, style, target_lyrics, annot_kws={}, title=None, version=None, **kwargs):
    solmization = solmize(stream, style, gamut=None)
    solmization.annotate(target_lyrics=target_lyrics, **annot_kws)
    eval = solmization.evaluate(target_lyrics=target_lyrics)
    report = process_evaluation(eval)
    if version is None: 
        now = datetime.now().strftime("%d-%m-%Y")
        version = f'Generated on {now}'
    metadata = dict(title=title, composer=version)
    stream.metadata = Metadata(lyricist=report, **metadata)
    return stream, solmization, eval

In [ ]:
def batch_process(source_files, target_dir, processor, refresh=False, version=None, remove_musicxml=True):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    evaluation = dict(
        date = datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        num_works = len(source_files),
        errors = {},
        works = {}
    )
    for source_fn in source_files:
        id = os.path.basename(source_fn).split('.')[0]
        target_pdf = f'{target_dir}/{id}.pdf'
        target_musicxml = f'{target_dir}/{id}.musicxml'
        if not os.path.exists(target_pdf) or refresh:
            try:
                score, _, eval = processor(source_fn, version=version)
                score.write('musicxml.pdf', target_pdf)
                if remove_musicxml:
                    os.remove(target_musicxml)
                evaluation['works'][id] = eval
            except Exception as e:
                evaluation['errors'][id] = str(e)
                print('An error occurred processing', source_fn)
    
    evaluation['num_errors'] = len(evaluation['errors'])
    with open(f'{target_dir}/evaluation.yaml', 'w') as yaml_file:
        yaml.dump(evaluation, yaml_file)


In [ ]:
def load_evaluation(yaml_fn, print_results=True, percentiles = [1, .95, .9, .85, .8]):
    with open(yaml_fn, 'r') as yaml_file:
        evaluation = yaml.safe_load(yaml_file)
    df = pd.DataFrame(evaluation['works']).T.fillna(0)
    df['perc_incorrect'] = df['incorrect'] / df['num_notes']
    df['perc_missing'] = df['missing'] / df['num_notes']

    if print_results:
        total_notes =  df['num_notes'].sum()
        total_correct = df['correct'].sum()
        total_errors = total_notes - total_correct
        print(f'Total number of notes: {total_notes}')
        print(f'Total number of correct notes: {total_correct} ({total_correct / total_notes:.0%})')
        print(f'Total number of errors: {total_errors} ({total_errors/total_notes:.0%})')
        for percentile in percentiles:
            count = sum(df['accuracy'] >= percentile)
            print(f'Works with >= {percentile*100}% accuracy: {count} ({count/len(df):.0%})')

    return df, evaluation

### Marot and de Beze

In [ ]:
def process_MdB_work(source_fn, version=None):
    id = os.path.basename(source_fn).split('.')[0]
    score = music21.converter.parse(source_fn)
    return process_stream(
        score, 
        style='continental',
        target_lyrics=2,
        annot_kws = dict(lyric_offset=3, grey_lyrics_num=1),
        title=f'MdB — Psalm {id[3:]}',
        version=version
    )

# source_fn = f"{MdB_DIR}/musicxml/MdB018.musicxml"
# score, _, eval = process_MdB_work(source_fn)

In [ ]:
mdb_meta = yaml.safe_load(open(f'{MdB_DIR}/collection.yaml', 'r'))
mdb_files = []
for work in mdb_meta['works']:
    mdb_files.append(f"{MdB_DIR}/musicxml/{work['id']}.musicxml")

batch_process(
    source_files = mdb_files, 
    target_dir = f'{MdB_DIR}/evaluation', 
    processor=process_MdB_work,
    refresh=False)

In [ ]:
df, evaluation = load_evaluation(f'{MdB_DIR}/evaluation/evaluation.yaml')
df.describe()

In [ ]:
sns.violinplot(data=df[['accuracy', 'perc_missing', 'perc_incorrect']], inner='point', cut=0)

### Sternhold and Hopkins

In [ ]:
def process_SH_work(source_fn, version=None):
    id = os.path.basename(source_fn).split('.')[0]
    score = music21.converter.parse(source_fn)
    return process_stream(
        score, 
        style='english',
        target_lyrics=2,
        annot_kws = dict(lyric_offset=3, grey_lyrics_num=1),
        title=f'SH — Psalm {id[2:]}',
        version=version
    )

# source_fn = f'{SH_DIR}/musicxml/SH001.musicxml'
# score, _, eval = process_SH_work(source_fn)

In [ ]:
sh_meta = yaml.safe_load(open(f'{SH_DIR}/collection.yaml', 'r'))
sh_files = []
for works in sh_meta['works']:
    sh_files.append(f"{SH_DIR}/musicxml/{works['id']}.musicxml")

batch_process(
    source_files = sh_files, 
    target_dir = f'{SH_DIR}/evaluation', 
    processor=process_SH_work,
    refresh=True)

In [ ]:
sh_df, sh_evaluation = load_evaluation(f'{SH_DIR}/evaluation/evaluation.yaml')
sh_df.describe()

In [ ]:
sh_df

In [ ]:
sns.violinplot(data=sh_df[['accuracy', 'perc_missing', 'perc_incorrect']], inner='point', cut=0)

### Smith

In [ ]:
def process_smith_work(source_fn, version=None):
    score = music21.converter.parse(source_fn)
    id = os.path.basename(source_fn).split('.')[0]
    piece, voice = id.split('-')
    return process_stream(
        score, 
        style='continental',
        target_lyrics=1,
        annot_kws = dict(lyric_offset=3),
        title=f'{piece.title()} {voice.lower()}',
        version=version
    )

# source_fn = f'{SMITH_DIR}/musicxml/goudimel-Alto.musicxml'
# score, _, eval = process_smith_work(source_fn)

In [ ]:
smith_meta = yaml.safe_load(open(f'{SMITH_DIR}/collection.yaml', 'r'))
smith_files = []
for works in smith_meta['works']:
    smith_files.append(f"{SMITH_DIR}/musicxml/{works['id']}.musicxml")

batch_process(
    source_files = smith_files, 
    target_dir = f'{SMITH_DIR}/evaluation', 
    processor=process_smith_work,
    remove_musicxml=False,
    refresh=True)

In [ ]:
smith_df, smith_evaluation = load_evaluation(f'{SMITH_DIR}/evaluation/evaluation.yaml')
smith_df.describe()

In [ ]:
smith_df

In [ ]:
goudimel_fn = f'{SMITH_DIR}/musicxml/goudimel.musicxml'
isaac_fn = f'{SMITH_DIR}/musicxml/isaac.musicxml'
smith_files = dict(goudimel=goudimel_fn, isaac= isaac_fn)
target_dir = f'{SMITH_DIR}/evaluation', 
for name, source_fn in smith_files.items():
    score = music21.converter.parse(source_fn)
    evaluations = []
    for part in score.parts:
        try: 
            solmization = solmize(part, style="continental")
            solmization.annotate(target_lyrics=1, best_only=True)
            evaluations.append(solmization.evaluate(target_lyrics=1))
        except Exception as e:
            print('skipping part...', part, e)

    target_pdf = f'{SMITH_DIR}/evaluation/{name}.pdf'
    score.write('musicxml.pdf', target_pdf)
    os.remove(f'{SMITH_DIR}/evaluation/{name}.musicxml')

### **TO DO** Other collectionsdp

In [ ]:
target_dir = 'evaluation'
refresh = True

sources = glob.glob(f'{CORPUS_DIR}/**')
for source_dir in sources[2:3]:
    files = sorted(glob.glob(f'{source_dir}/musicxml/*.musicxml'))
    if not os.path.exists(f'{source_dir}/{target_dir}'):
        os.makedirs(f'{source_dir}/{target_dir}')
    for file in files:
        name, _ =  os.path.basename(file).split('.')
        target_fn = f'{source_dir}/{target_dir}/{name}.pdf'
        if not os.path.exists(target_fn) or refresh:
            try:
                score = converter.parse(file)
                evaluateScore(score, onlyBest=False)
                score.write('musicxml.pdf', target_fn)
            except Exception as e:
                print('Skipped', file)
        